# Webscrapen van HTML

HTML (HyperTextMarkupLanguage) is de opmaak van een webpagina.  
De HTML kan dynamisch worden opgemaakt met (bijvoorbeeld) Javascipt code.  
Maar HTML paginas kunnen ook statisch zijn.  
Een statische pagina moet ververst worden om nieuwe data te zien.  

De server die de paginas naar de client stuurt moet wel weten wat het moet doen browser (_client_) een pagina opvraagt.  
Als dit niet via JSON of speciale headers gebeurd dan wordt het commando beschreven in de URL zelf.  
Dit gebeurd doormiddel van een query.  

Een query is de gedeelte in een URL na de `?`.  
De query bestaat uit een key en optioneel een value.  
Key en value worden aan elkaar gekoppeld doormiddel van een `=`.  
Er kunnen ook meerdere queries in een url als de queries gesepereerd worden door een `&`.  
De keys en values hoeven niet uniek te zijn.  

Voorbeeld van een valide URL:  
`https://www.web.site/path.html?key=value&key=value&only_key`

De query kan handmatig veranderd worden om de website te manipuleren.  
Zo kan er data worden opgevraagd die normaal niet aanwezig is op de webpagina.  

Open een browser en de Developer Tools.  
Navigeer naar de demo pagina https://computer-database.gatling.io/computers  
Dit is een demo website van [Gatling] en gebruikt URL queries.  

[Gatling]: https://gatling.io/

>![computer database home page url](./img/computerdatabase_url_home_page.png)

Als we de URL bekijken zien we vrij weinig wat te manipuleren is maar dit is meestal het geval met een homepage.  
Dit kan al snel veranderen als we op de `next` knop klikken die aanwezig is in de pagina.  

> ![computer database next page url](./img/computerdatabase_url_next_page.png)

Nu heeft de browser een pagina met een andere inhoud.  
Ook is de URL verandered, de URL heeft een _query_.

In de developer tools open kan er worden gezien dat er een GET request is geweest.  
Ook kan de _parameters_ van de query worden ingezien.

> ![query string params](./img/devtools_query_params.png "Query String Parameters")

Met wat giswerk kan er worden bedacht wat de query bevat.

`p=0` de key `p` en de value als cijfer moet waarschijnlijk de pagina voorstellen.  
`n=10` de key `n` is hoogstwaarschijnlijk het aantal items in de [table] op de pagina.  
`s=name` de table is gesorteerd op naam, dus `s` is voor _sort_.  
`d=asc` moet dan voor de manier van sorteren zijn, `asc` staat voor _ascending_.

[table]: https://developer.mozilla.org/en-US/docs/Learn/HTML/Tables/Basics

Met deze kennis kan er een query gemaakt worden voor een tabel met meer dan 10 regels.  
Zo hoeft er minder op de `next` knop gedrukt worden om de database van de website te scrapen. 

https://computer-database.gatling.io/computers?p=0&n=100&s=name&d=desc  
Als deze URL wordt bezocht kan er een tabel worden gezien met 100 entries.

---

# HTML pagina scrapen met Python

Nu de API van de website is gevonden kan deze zelfde data worden opgevraagd via Python.  
Dit kan worden gedaan met de third-party library: [requests].  
De HTML die requests terug geeft kan worden geparsed met [beautifulsoup4].  

De libraries moet geinstalleerd worden met `pip`  

[requests]: https://docs.python-requests.org/en/master/index.html
[beautifulsoup4]: https://www.crummy.com/software/BeautifulSoup/

In [1]:
!python -m pip install --upgrade pip requests beautifulsoup4

Importeer `requests` en de _class_ `BeautifulSoup` uit de module `bs4`  
De functie `urlencode` uit de `urllib.parse` library helpt ons de queries creeeren.

In [2]:
from urllib.parse import urlencode

import requests
from bs4 import BeautifulSoup

In [3]:
base_url = 'https://computer-database.gatling.io/computers?'

query  = {"p": 0, "n": 100}
url_query = urlencode(query)
url = base_url + url_query
url

'https://computer-database.gatling.io/computers?p=0&n=100'

De URL is gecreeerd en kan dus worden opgevraagd door requests.  
Het attribute `content` van het object `response` is de HTML data in `bytes`.  
Deze HTML data geven we door aan `BeautifulSoup`.  
Deze _class_ parsed de data zodat er gemakkelijker elementen van de pagina kan worden opgevraagd.  

In [4]:
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# vraag de title Tag op van de HTML data
soup.title

<title>Computers database</title>

De website laat zien dat er een table op de pagina aanwezig is.  
De data uit deze table halen is het doel.

In [5]:
from IPython import display

table = soup.table
# display.HTML(str(table))  # uncomment om de table te laten zien in de output

Met een [list-comprehension] kan er  over de `<th>` tags geitereerd worden en zo uit elke Tag de text filteren.  
Een list-comprehension is een loop in een data container zoals een `list`.  
De loop itereert en plaatst te objecten in een nieuwe data-container.  
list-comprehensions zijn efficient en flexibel.

[list-comprehension]: https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions

In [6]:
headers = [h.get_text() for h in table.thead.find_all('th')]
headers  # list met str objecten

['Computer name', 'Introduced', 'Discontinued', 'Company']

Nu er een lijst met headers is is er een indicatie welke cel wat voor data bevat.  
De headers kan gebruikt worden als _key_ in een dict.  
De cellen in de regels zijn dan de _values_.  



In [7]:
rows = table.tbody.find_all('tr')

# laat de eerste 5 regels zien van het tabel
rows[0:5]  # list met bs4.element.Tag objecten

[<tr><td><a href="/computers/381">ACE</a></td><td>-</td><td>-</td><td>-</td></tr>,
 <tr><td><a href="/computers/501">AN/FSQ-32</a></td><td>01 Jan 1960</td><td>-</td><td>IBM</td></tr>,
 <tr><td><a href="/computers/500">AN/FSQ-7</a></td><td>01 Jan 1958</td><td>-</td><td>IBM</td></tr>,
 <tr><td><a href="/computers/388">APEXC</a></td><td>-</td><td>-</td><td>-</td></tr>,
 <tr><td><a href="/computers/355">ARRA</a></td><td>-</td><td>-</td><td>-</td></tr>]

De headers en zijn de regels geparsed uit de table van de website.  
Nu kan deze data in een CSV file of JSON gezet worden.

Hieronder een voorbeeld hoe er een JSON van de data gemaakt kan worden.

In [8]:
import json

row_list = []

for row in rows:
    row_text = [r.get_text() for r in row]  # list comprehension om een list met text te maken
    # dict-zip
    header_row_dict = dict(zip(headers, row_text))
    row_list.append(header_row_dict)


# creeer de json
json_payload = json.dumps({'rows': row_list}, indent=2)

# print(json_payload)

Hieronder een voorbeeld hoe de er een CSV file met de data gemaakt kan worden.

In [9]:
import csv

filename = 'computer_database.txt'

# `with` context manager
with open(filename, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    
    writer.writerow(headers)  # schrijf de header regel (fieldnames)
    for row in rows:
        row_text = [r.get_text() for r in row]
        writer.writerow(row_text)  # schrijf de regels
        
# print(open(filename, 'r').read())